# Estudo de Self-Attention

### Gerando Inputs

In [1]:
import torch
import numpy as np

Sentenças Fictícias:

- o mikas ta ali
- eu gosto de batata frita
- existem elefantes

Informações
- O tamanho da maior sentença é 5.
- Temos 3 exemplos de teste.
- Usaremos inteiros para representar cada palavra.
- Usaremos 0 para representar os espaços vazios das sentenças.
- O tamanho do nosso vocabulário é 12 pois temos 12 palavras distintas.


In [2]:
sentence_matrix = torch.LongTensor([
    [1,   2,  3,  4, 0], #       o     mikas   ta      ali
    [5,   6,  7,  8, 9], #      eu     gosto   de   batata   frita
    [10, 11,  0,  0, 0], # existem elefantes

])
qnt_sentences, max_sentence_len = sentence_matrix.shape
print(f"Temos {qnt_sentences} sentenças e a maior sentença tem {max_sentence_len} tokens.")
sentence_matrix.shape

Temos 3 sentenças e a maior sentença tem 5 tokens.


torch.Size([3, 5])

Em geral não usamos representações inteiras para representar palavras, mas sim embedding.

Geraremos embeddings fictícios de tamanho 10 para os 12 tokens disponíveis no vocabulário.

In [3]:
vocab_size = len(sentence_matrix.unique())
embedding_size = 10
embeddings = np.random.random((vocab_size, embedding_size))
# Convertendo para tensor:
embeddings = torch.Tensor(embeddings)
print(f"Temos {max_sentence_len} embeddings de tamanho {embedding_size}.")

Temos 5 embeddings de tamanho 10.


Agora geraremos a matriz de input para os modelos.

- Nela teremos o shape 3, 5, 10 que representam as 3 sentenças com no máximo 5 tokens.
- Cada token é representado por um embedding de tamanho 10.

In [4]:
indexes = sentence_matrix
input_matrix = embeddings[indexes]
print("Temos {0} instancias de no máximo {1} tokens representados por embeddings de tamanho {2}.".format(*input_matrix.shape))
input_matrix.shape

Temos 3 instancias de no máximo 5 tokens representados por embeddings de tamanho 10.


torch.Size([3, 5, 10])

### Passando inputs pela camada de RNN:

A atenção é gerada a partir dos outputs gerados pela camada da RNN, portanto passaremos o input pela camada de RNN:

A nova saída da RNN vai ter shape (3, 5, 100) onde 3 é o número de batches, 5 é o tamanho máximo da sequência e 4 é o tamanho da representação gerada pela RNN (hidden_size).

In [5]:
rnn = torch.nn.RNN(input_size=embedding_size, hidden_size=4, batch_first=True)
output, _ = rnn(input_matrix)
print("Temos {0} inputs de tamanho máximo {1} representado por {2} dimensões.".format(*output.shape))

Temos 3 inputs de tamanho máximo 5 representado por 4 dimensões.


In [6]:
output.shape

torch.Size([3, 5, 4])

In [7]:
linear = torch.nn.Linear(4, 5, bias=False)
linear_output = linear(output)

In [8]:
print(linear_output.shape)
linear_output

torch.Size([3, 5, 5])


tensor([[[ 0.4493, -0.8465, -0.4649, -0.7994,  0.2040],
         [ 0.3613, -0.8698, -0.4298, -0.7632,  0.2384],
         [ 0.2638, -0.8425, -0.3597, -0.6367,  0.2164],
         [ 0.1620, -0.6044, -0.3500, -0.5218,  0.3112],
         [ 0.3678, -0.9263, -0.4137, -0.7505,  0.1974]],

        [[ 0.5426, -1.0014, -0.4813, -0.8695,  0.1229],
         [ 0.2917, -0.7588, -0.4266, -0.6434,  0.2789],
         [ 0.0886, -0.6661, -0.3225, -0.5285,  0.3490],
         [ 0.4143, -0.7720, -0.5061, -0.6674,  0.2437],
         [ 0.1935, -0.5650, -0.4226, -0.4285,  0.3245]],

        [[ 0.3615, -0.7878, -0.4019, -0.5959,  0.1527],
         [ 0.2628, -0.6280, -0.4005, -0.5332,  0.2600],
         [ 0.3425, -0.9095, -0.4020, -0.7436,  0.2118],
         [ 0.4627, -0.9608, -0.4652, -0.8581,  0.1925],
         [ 0.4504, -0.9553, -0.4612, -0.8542,  0.2005]]],
       grad_fn=<UnsafeViewBackward>)

In [9]:
torch.nn.functional.softmax(linear_output, dim=-1)

tensor([[[0.3644, 0.0997, 0.1461, 0.1046, 0.2852],
         [0.3385, 0.0988, 0.1534, 0.1099, 0.2993],
         [0.3099, 0.1025, 0.1661, 0.1259, 0.2955],
         [0.2681, 0.1246, 0.1607, 0.1353, 0.3113],
         [0.3446, 0.0945, 0.1577, 0.1126, 0.2906]],

        [[0.4043, 0.0863, 0.1452, 0.0985, 0.2657],
         [0.3108, 0.1087, 0.1516, 0.1220, 0.3069],
         [0.2519, 0.1184, 0.1670, 0.1359, 0.3268],
         [0.3465, 0.1058, 0.1380, 0.1175, 0.2922],
         [0.2714, 0.1271, 0.1465, 0.1457, 0.3093]],

        [[0.3358, 0.1064, 0.1565, 0.1289, 0.2725],
         [0.2964, 0.1216, 0.1527, 0.1337, 0.2956],
         [0.3360, 0.0961, 0.1596, 0.1134, 0.2949],
         [0.3750, 0.0903, 0.1483, 0.1001, 0.2862],
         [0.3708, 0.0909, 0.1490, 0.1006, 0.2888]]], grad_fn=<SoftmaxBackward>)